In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
data = pd.read_csv("./train.csv")
data.head()

,Label,F0(pitch),F1,F2,F3,F4,F5
0,BYJ,NaN,934.456137,1867.766912,2958.658056,4026.961381,NaN
1,BYJ,NaN,1338.936874,2694.203872,2875.547156,4229.574451,NaN
2,BYJ,NaN,1160.135803,2144.572867,3476.990084,4433.055683,NaN
3,BYJ,NaN,211.203260,1472.209366,2708.645128,3839.022388,NaN
4,BYJ,NaN,381.149933,1721.053643,2682.587186,3957.900682,NaN


In [3]:
print("라벨:", data["Label"].unique(), sep="\n")

라벨:
['BYJ' 'HYN' 'SJW']


In [ ]:
#3명이 7개의 피쳐에 대해 어떤 분포를 가지고 있는지 시각화
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(data, hue="Label", palette="husl")

### preprocess

In [4]:
import tensorflow as tf
import os
import datetime
from sklearn.utils import shuffle

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [5]:
train_data = data.fillna(value=0)
user_id_arr = train_data['Label'].unique()
user_count = user_id_arr.shape[0]
train_data = pd.get_dummies(train_data)
train_data = train_data.astype('float64')
train_data = shuffle(train_data)
X_len=len(train_data.columns)-user_count
y_len=user_count
X=train_data.iloc[:,0:X_len]
y=train_data.iloc[:,-user_count:]

In [6]:
X.head()

,F0(pitch),F1,F2,F3,F4,F5
576,0.000000,1956.155406,2710.009480,3816.408405,4409.109038,0.000000
445,0.000000,914.393773,2213.054230,3297.138564,4341.879894,0.000000
484,0.000000,918.327711,2040.409655,3128.793551,4304.133192,0.000000
150,0.000000,635.351407,1792.649081,2465.669464,4044.486379,0.000000
508,196.552652,387.393985,2163.231315,2968.388701,3673.923567,4831.864733


In [7]:
y.head()

,Label_BYJ,Label_HYN,Label_SJW
576,0.0,0.0,1.0
445,0.0,0.0,1.0
484,0.0,0.0,1.0
150,1.0,0.0,0.0
508,0.0,0.0,1.0


### train

In [8]:
#ready model

k=20 #fold
num_val_samples=len(X)//20

sc = StandardScaler()
classifier = Sequential()

unit = 15
rate = 0.3

# 1st layer(input)
classifier.add(Dense(units = unit, kernel_initializer = 'glorot_uniform', activation = 'relu', input_dim = X_len))

# 2nd layer
classifier.add(Dense(units = unit, kernel_initializer = 'glorot_uniform', activation = 'relu'))
classifier.add(Dropout(rate))

# 3rd layer
classifier.add(Dense(units = unit, kernel_initializer = 'glorot_uniform', activation = 'relu'))
classifier.add(Dropout(rate))

# 4th layer
classifier.add(Dense(units = unit, kernel_initializer = 'glorot_uniform', activation = 'relu'))
classifier.add(Dropout(rate))

# 5th layer(output)
classifier.add(Dense(units = user_count, kernel_initializer = 'glorot_uniform', activation = 'softmax'))

# compile
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                105       
_________________________________________________________________
dense_1 (Dense)              (None, 15)                240       
_________________________________________________________________
dropout (Dropout)            (None, 15)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 15)                240       
_________________________________________________________________
dropout_1 (Dropout)          (None, 15)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 15)                240       
_________________________________________________________________
dropout_2 (Dropout)          (None, 15)                0

In [9]:
#fitting and validation through cross validation

for i in range(k):
    print('fold num #',i+1)
    
    #validation data
    val_data=X[i*num_val_samples:(i+1)*num_val_samples]
    val_targets=y[i*num_val_samples:(i+1)*num_val_samples]
    
    #ready train data and target data
    partial_train_data=np.concatenate([X[:i*num_val_samples], X[(i+1)*num_val_samples:]],axis=0)
    partial_train_targets=np.concatenate([y[:i*num_val_samples], y[(i+1)*num_val_samples:]],axis=0)
    
    #Feature Scaling
    partial_train_data=sc.fit_transform(partial_train_data)
    val_data=sc.transform(val_data)
    
    #fitting
    classifier.fit(partial_train_data, partial_train_targets, batch_size = 40, epochs = 40)

fold num # 1
Epoch 1/40
16/16 [==============================] - 0s 999us/step - loss: 1.1029 - accuracy: 0.3494
Epoch 2/40
16/16 [==============================] - 0s 1ms/step - loss: 1.0913 - accuracy: 0.3782
Epoch 3/40
16/16 [==============================] - 0s 1ms/step - loss: 1.0777 - accuracy: 0.3574
Epoch 4/40
16/16 [==============================] - 0s 961us/step - loss: 1.0665 - accuracy: 0.3734
Epoch 5/40
16/16 [==============================] - 0s 1ms/step - loss: 1.0757 - accuracy: 0.4006
Epoch 6/40
16/16 [==============================] - 0s 1ms/step - loss: 1.0700 - accuracy: 0.4407
Epoch 7/40
16/16 [==============================] - 0s 1ms/step - loss: 1.0613 - accuracy: 0.4391
Epoch 8/40
16/16 [==============================] - 0s 1ms/step - loss: 1.0644 - accuracy: 0.4151
Epoch 9/40
16/16 [==============================] - 0s 1000us/step - loss: 1.0633 - accuracy: 0.4279
Epoch 10/40
16/16 [==============================] - 0s 936us/step - loss: 1.0572 - accuracy: 0.43

16/16 [==============================] - 0s 1ms/step - loss: 0.7379 - accuracy: 0.6971
Epoch 4/40
16/16 [==============================] - 0s 1ms/step - loss: 0.7307 - accuracy: 0.6923
Epoch 5/40
16/16 [==============================] - 0s 1ms/step - loss: 0.7402 - accuracy: 0.6699
Epoch 6/40
16/16 [==============================] - 0s 1ms/step - loss: 0.7523 - accuracy: 0.6939
Epoch 7/40
16/16 [==============================] - 0s 1ms/step - loss: 0.7643 - accuracy: 0.6795
Epoch 8/40
16/16 [==============================] - 0s 1ms/step - loss: 0.7409 - accuracy: 0.7019
Epoch 9/40
16/16 [==============================] - 0s 1ms/step - loss: 0.7356 - accuracy: 0.7003
Epoch 10/40
16/16 [==============================] - 0s 1ms/step - loss: 0.7108 - accuracy: 0.7212
Epoch 11/40
16/16 [==============================] - 0s 1ms/step - loss: 0.7260 - accuracy: 0.7003
Epoch 12/40
16/16 [==============================] - 0s 1ms/step - loss: 0.7590 - accuracy: 0.6811
Epoch 13/40
16/16 [=========

16/16 [==============================] - 0s 898us/step - loss: 0.6734 - accuracy: 0.7115
Epoch 5/40
16/16 [==============================] - 0s 1000us/step - loss: 0.6590 - accuracy: 0.7324
Epoch 6/40
16/16 [==============================] - 0s 1ms/step - loss: 0.6540 - accuracy: 0.7260
Epoch 7/40
16/16 [==============================] - 0s 957us/step - loss: 0.6824 - accuracy: 0.7147
Epoch 8/40
16/16 [==============================] - 0s 1ms/step - loss: 0.6756 - accuracy: 0.7147
Epoch 9/40
16/16 [==============================] - 0s 984us/step - loss: 0.6749 - accuracy: 0.7067
Epoch 10/40
16/16 [==============================] - 0s 1ms/step - loss: 0.6458 - accuracy: 0.7308
Epoch 11/40
16/16 [==============================] - 0s 867us/step - loss: 0.6864 - accuracy: 0.7452
Epoch 12/40
16/16 [==============================] - 0s 884us/step - loss: 0.6510 - accuracy: 0.7340
Epoch 13/40
16/16 [==============================] - 0s 983us/step - loss: 0.6991 - accuracy: 0.7196
Epoch 14/40


16/16 [==============================] - 0s 1ms/step - loss: 0.6352 - accuracy: 0.7436
Epoch 8/40
16/16 [==============================] - 0s 1ms/step - loss: 0.6325 - accuracy: 0.7548
Epoch 9/40
16/16 [==============================] - 0s 1ms/step - loss: 0.6415 - accuracy: 0.7612
Epoch 10/40
16/16 [==============================] - 0s 1ms/step - loss: 0.6535 - accuracy: 0.7372
Epoch 11/40
16/16 [==============================] - 0s 1ms/step - loss: 0.6861 - accuracy: 0.7324
Epoch 12/40
16/16 [==============================] - 0s 1ms/step - loss: 0.6297 - accuracy: 0.7548
Epoch 13/40
16/16 [==============================] - 0s 1ms/step - loss: 0.6435 - accuracy: 0.7468
Epoch 14/40
16/16 [==============================] - 0s 1ms/step - loss: 0.6465 - accuracy: 0.7276
Epoch 15/40
16/16 [==============================] - 0s 1ms/step - loss: 0.6373 - accuracy: 0.7340
Epoch 16/40
16/16 [==============================] - ETA: 0s - loss: 0.7004 - accuracy: 0.72 - 0s 1ms/step - loss: 0.6609 -

16/16 [==============================] - 0s 733us/step - loss: 0.6315 - accuracy: 0.7404
Epoch 10/40
16/16 [==============================] - 0s 734us/step - loss: 0.6160 - accuracy: 0.7404
Epoch 11/40
16/16 [==============================] - 0s 751us/step - loss: 0.6249 - accuracy: 0.7420
Epoch 12/40
16/16 [==============================] - 0s 805us/step - loss: 0.5880 - accuracy: 0.7548
Epoch 13/40
16/16 [==============================] - 0s 867us/step - loss: 0.5986 - accuracy: 0.7500
Epoch 14/40
16/16 [==============================] - 0s 811us/step - loss: 0.6215 - accuracy: 0.7612
Epoch 15/40
16/16 [==============================] - 0s 710us/step - loss: 0.5993 - accuracy: 0.7628
Epoch 16/40
16/16 [==============================] - 0s 733us/step - loss: 0.6357 - accuracy: 0.7452
Epoch 17/40
16/16 [==============================] - 0s 781us/step - loss: 0.6643 - accuracy: 0.7340
Epoch 18/40
16/16 [==============================] - 0s 733us/step - loss: 0.6391 - accuracy: 0.7244
Ep

### test - SJW ###

In [131]:
test_data = pd.read_csv("./test_SJW.csv")
test_data = pd.get_dummies(test_data)
test_data['Label_HYN'] = 0.0
test_data['Label_BYJ'] = 0.0

test_data = test_data.fillna(value=0.0)
test_data = test_data.astype('float64')
test_data = shuffle(test_data)

user_count = 3
X_len=len(test_data.columns)-user_count
y_len=user_count

X=test_data.iloc[:,0:X_len]
y=test_data.iloc[:,-y_len:]

test_pred=classifier.predict(X)
test_pred=test_pred.tolist()
num = len(test_pred[0])
test_pred_df = pd.DataFrame(test_pred,columns=['Label_HYN', 'Label_BYJ', 'Label_SJW'])

print("--------------------------------------------------------")
print("                 BYJ, HYN인 척 하는 SJW                  ")
print("실제로 BYJ:", y["Label_BYJ"].unique(), 
      "실제로 HYN:", y["Label_HYN"].unique(), 
      "실제로 SJW:", y["Label_SJW"].unique(), sep="\n")
print("--------------------------------------------------------")
print("               실제 AI 모델의 판단 결과                 ")
print("BYJ으로 판단:", test_pred_df["Label_BYJ"].unique(), 
      "HYN으로 판단:", test_pred_df["Label_HYN"].unique(), 
      "SJW으로 판단:", test_pred_df["Label_SJW"].unique(), sep="\n")
print("--------------------------------------------------------")

test_pred_df.to_csv('./predict_SJW.csv')

--------------------------------------------------------
                 BYJ, HYN인 척 하는 SJW                  
실제로 BYJ:
[0.]
실제로 HYN:
[0.]
실제로 SJW:
[1.]
--------------------------------------------------------
               실제 AI 모델의 판단 결과                 
BYJ으로 판단:
[0.]
HYN으로 판단:
[0.]
SJW으로 판단:
[1.]
--------------------------------------------------------


### test - HYN ###

In [132]:
test_data = pd.read_csv("./test_HYN.csv")
test_data = pd.get_dummies(test_data)
test_data['Label_BYJ'] = 0.0
test_data['Label_SJW'] = 0.0

test_data = test_data.fillna(value=0.0)
test_data = test_data.astype('float64')
test_data = shuffle(test_data)

user_count = 3
X_len=len(test_data.columns)-user_count
y_len=user_count

X=test_data.iloc[:,0:X_len]
y=test_data.iloc[:,-y_len:]

test_pred=classifier.predict(X)
test_pred=test_pred.tolist()
num = len(test_pred[0])
test_pred_df = pd.DataFrame(test_pred,columns=['Label_BYJ', 'Label_SJW', 'Label_HYN'])

print("--------------------------------------------------------")
print("                 BYJ, SJW인 척 하는 HYN                  ")
print("실제로 BYJ:", y["Label_BYJ"].unique(), 
      "실제로 HYN:", y["Label_HYN"].unique(), 
      "실제로 SJW:", y["Label_SJW"].unique(), sep="\n")
print("--------------------------------------------------------")
print("               실제 AI 모델의 판단 결과                 ")
print("BYJ으로 판단:", test_pred_df["Label_BYJ"].unique(), 
      "HYN으로 판단:", test_pred_df["Label_HYN"].unique(), 
      "SJW으로 판단:", test_pred_df["Label_SJW"].unique(), sep="\n")
print("--------------------------------------------------------")

test_pred_df.to_csv('./predict_HYN.csv')

--------------------------------------------------------
                 BYJ, SJW인 척 하는 HYN                  
실제로 BYJ:
[0.]
실제로 HYN:
[1.]
실제로 SJW:
[0.]
--------------------------------------------------------
               실제 AI 모델의 판단 결과                 
BYJ으로 판단:
[0.]
HYN으로 판단:
[1.]
SJW으로 판단:
[0.]
--------------------------------------------------------


### test - BYJ ###

In [134]:
test_data = pd.read_csv("./test_BYJ.csv")
test_data = pd.get_dummies(test_data)
test_data['Label_SJW'] = 0.0
test_data['Label_HYN'] = 0.0

test_data = test_data.fillna(value=0.0)
test_data = test_data.astype('float64')
test_data = shuffle(test_data)

user_count = 3
X_len=len(test_data.columns)-user_count
y_len=user_count

X=test_data.iloc[:,0:X_len]
y=test_data.iloc[:,-y_len:]

test_pred=classifier.predict(X)
test_pred=test_pred.tolist()
num = len(test_pred[0])
test_pred_df = pd.DataFrame(test_pred,columns=['Label_SJW', 'Label_HYN', 'Label_BYJ'])

print("--------------------------------------------------------")
print("                 SJW, HYN인 척 하는 BYJ                  ")
print("실제로 BYJ:", y["Label_BYJ"].unique(), 
      "실제로 HYN:", y["Label_HYN"].unique(), 
      "실제로 SJW:", y["Label_SJW"].unique(), sep="\n")
print("--------------------------------------------------------")
print("               실제 AI 모델의 판단 결과                 ")
print("BYJ으로 판단:", test_pred_df["Label_BYJ"].unique(), 
      "HYN으로 판단:", test_pred_df["Label_HYN"].unique(), 
      "SJW으로 판단:", test_pred_df["Label_SJW"].unique(), sep="\n")
print("--------------------------------------------------------")

test_pred_df.to_csv('./predict_BYJ.csv')

--------------------------------------------------------
                 SJW, HYN인 척 하는 BYJ                  
실제로 BYJ:
[1.]
실제로 HYN:
[0.]
실제로 SJW:
[0.]
--------------------------------------------------------
               실제 AI 모델의 판단 결과                 
BYJ으로 판단:
[1.]
HYN으로 판단:
[0.]
SJW으로 판단:
[0.]
--------------------------------------------------------
